In [1]:
### THIS SCRIPT CLEANS THE COLLECTED SEP ###   

In [2]:
import pymongo 
import requests
import json
import pandas as pd
import re

from tqdm import tqdm
from bson.objectid import ObjectId
from pprint import pprint

import lib_sepinpho as sep

##### INIT GLOBAL VARIABLES#####

#init Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#create database
db = client.visualizing_sep

In [3]:
##### Cleaning Sep_Entries ########
##### Fall 2020 Edition: Completed 10.1.2020 ########
##### Spring 2021 Edition: No Updates ########

#1. Check for entries with InPhO apis.

# collection_to_clean = db.sep_entries

# sep_no_inpho = pd.DataFrame(collection_to_clean.find( filter={'inpho_api': 'Error: No InPhO entry'},
#                                          projection = ['title'],
#                                          sort=[('title',1)]
#                                         ))
# #export to csv to made edits in this file
# sep_no_inpho.to_csv('noinpho.csv')


In [49]:
# read the values to update from the CSV and batch update the entries

# collection_to_update = db.sep_entries

# #import cleaned data
# inpho_updates = pd.read_csv('noinpho.csv')

# for index,row in tqdm(inpho_updates.iterrows()):
#     title = row['title']
#     api_endpoint = row['api_endpoint']
#     json_type = row['json_type']

#     sep.update_sep_json(title,api_endpoint, json_type, collection_to_update)



1it [00:00,  1.80it/s]acknowledged: http://inpho.cogs.indiana.edu/thinker/2553.json
 True
[{&#39;_id&#39;: ObjectId(&#39;5f1bafb6896f82cdbda60f27&#39;), &#39;title&#39;: &#39;Aristotle&#39;, &#39;inpho_api&#39;: &#39;http://inpho.cogs.indiana.edu/thinker/2553.json&#39;, &#39;inpho_json&#39;: {&#39;wiki&#39;: &#39;Aristotle&#39;, &#39;influenced&#39;: [2462, 2472, 2477, 2493, 2543, 2549, 2576, 2577, 2671, 2685, 2731, 2768, 2905, 2943, 2953, 2985, 3021, 3049, 3114, 3132, 3177, 3179, 3223, 3295, 3367, 3383, 3404, 3439, 3476, 3511, 3550, 3580, 3585, 3590, 3597, 3657, 3738, 3760, 3776, 3777, 3805, 3842, 3911, 3936, 3950, 3963, 3974, 3975, 3991, 4006, 4021, 4025, 4077, 4189], &#39;death&#39;: [{&#39;month&#39;: 3, &#39;day&#39;: 7, &#39;year&#39;: -322}], &#39;teachers&#39;: [3724], &#39;url&#39;: &#39;/thinker/2553&#39;, &#39;influenced_by&#39;: [3191, 3682, 3724, 3919], &#39;professions&#39;: [&#39;Student&#39;, &#39;Teacher&#39;], &#39;death_strings&#39;: [&#39;March 7, 322 B.C.E.&#39;], 

In [4]:
##### Cleaning Sep_Entries ########
##### Summer: Completed 7.16.2020 ########
##### Summer: Completed 10.1.2020 ########

#2. Check for duplicate InPhO JSON data.

collection_to_clean = db.sep_entries
result = list(collection_to_clean.aggregate([
                                {
                                    '$group' : { '_id' : "$inpho_json.ID",
                                    'count': { '$sum': 1 }
                                    }
                                },
                                {
                                    '$match': { 'count': {'$ne': 1 }}
                                }]))

pprint(result)

[{'_id': None, 'count': 13}]


In [5]:

#Find any article that doesn't have domain info

collection_to_update = db.sep_entries

no_domains = list(collection_to_update.find(
    {'primary_domain': {'$exists': False}},
    projection={'page_url':1, 'title':1, '_id':0}
))

pprint(no_domains)

[{'page_url': '/entries/feminism-argumentation/',
  'title': 'Feminist Perspectives on Argumentation'},
 {'page_url': '/entries/human-nature/', 'title': 'Human Nature'},
 {'page_url': '/entries/hyperintensionality/', 'title': 'Hyperintensionality'}]


In [6]:
#take the list of articles in no_domains, and make a list of the domain data to add for each of them. 

domains_to_update = [
    ('Feminist Philosophy', 'Feminist Philosophy, Philosophy of Language, Logic'),
    ('Philosophy of Mind', 'Philosophy of Mind', 'Philosophy of Biology'),
    ('Philosophy of Language', 'Philosophy of Language, Logic'),

]

#check to make sure domains_to_update and no_domains are the same size
print(len(no_domains))
print(len(domains_to_update))

3
3


In [7]:
#update articles in no_domains with new domain data  
#Fall Update: Completed 10.1.2020#
#SPring 2021 Update: Completed 4.3.2021
collection_to_update = db.sep_entries

for i, articles in enumerate(no_domains):
    article_to_update = articles['page_url']
    primary_domain = domains_to_update[i][0]
    domain_tags = domains_to_update[i][1] 
    new_values = {
         'primary_domain': primary_domain,
         'domain_tags': domain_tags
         }
    result = collection_to_update.update_one(
        {'page_url': article_to_update},
        {'$set': new_values}
    )
    # boolean confirmation that the API call went through
    print (f'acknowledged: {article_to_update}\n', result.acknowledged)
    


acknowledged: /entries/feminism-argumentation/
 True
acknowledged: /entries/human-nature/
 True
acknowledged: /entries/hyperintensionality/
 True


In [8]:
#now test again for no domain info

collection_to_update = db.sep_entries

no_domains = list(collection_to_update.find(
    {'primary_domain': {'$exists': False}},
    projection={'page_url':1, 'title':1, '_id':0}
))

pprint(no_domains)

[]


In [13]:
#test for and remove editors notes
#Spring 2021 Update: I manually updated 8 articles in compass to remove the Editors Note from the preamble

collection_to_update = db.sep_entries

editors_notes = list(collection_to_update.find(
    {'preamble_text': {'$regex': '^\['}},
    projection={'page_url':1, 'title':1, 'preamble_text':1, '_id':0}
))
pprint(len(editors_notes))
# pprint(editors_notes)

for article in editors_notes:
    print(article['page_url'])

8
/entries/content-externalism/
/entries/evolution-before-darwin/
/entries/femapproach-continental/
/entries/hermeneutics/
/entries/origin-descent/
/entries/properties-emergent/
/entries/recursive-functions/
/entries/theory-bioethics/
